In [4]:
# questionnaire

In [ ]:
# age

data_array = []

while True:

	try:
		age = int(input('Enter your age: '))
		
		if 1 <= age <= 120:
			data_array.append(age)
			break
	except:
		
		print('Enter age as a number!')
	
# bmi

while True:

	try:
		bmi = float(input('Enter your bmi: '))
			
		if 10 <= bmi <= 40:
			data_array.append(bmi)
			break
	except:
		
		print('Enter bmi as a number!')

# children

while True:

	try:
		chld = int(input('How many children do you have: '))
			
		if 0 <= chld <= 30:
			data_array.append(chld)
			break
	except:
		
		print('Enter children amount as a number!')

# are you a smoker?

while True:

	smk = input('Are you a smoker (y/n): ')
	
	if smk in ['y', 'Y']:
		data_array.append(1)
		break
	elif smk in ['n', 'N']:
		data_array.append(0)
		break

# are you a male / female

while True:

	sex = input('Are you a male/female (m/f): ')
	
	if sex in ['m', 'M']:
		data_array.append(0)
		data_array.append(1)
		break
	elif sex in ['f', 'F']:
		data_array.append(1)
		data_array.append(0)
		break

# region - NE, NW, SE, SW

while True:

	print('Select your region.')
	print('1. north-east')
	print('2. north-west')
	print('3. south-east')
	print('4. south-west')
	
	reg = input('Your choice: ')
	
	if reg in '1, 2, 3, 4':
		
		if reg == '1':
			data_array.append(1)
			data_array.append(0)
			data_array.append(0)
			data_array.append(0)
			break
		elif reg == '2':
			data_array.append(0)
			data_array.append(1)
			data_array.append(0)
			data_array.append(0)
			break
		elif reg == '3':
			data_array.append(0)
			data_array.append(0)
			data_array.append(1)
			data_array.append(0)
			break
		elif reg == '4':
			data_array.append(0)
			data_array.append(0)
			data_array.append(0)
			data_array.append(1)
			break

Enter your age: 23


In [4]:
data_array

[25, 40.0, 4, 0, 0, 1, 1, 0, 0, 0]

In [6]:
import pandas as pd
import numpy as np

In [7]:
raw_data = pd.read_csv('insurance.csv')

In [8]:
raw_data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [9]:
raw_data.isna().sum()  # conclusion - dataset is complete

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [10]:
# smokers, sex, region

qualitative_columns = ['smoker', 'sex', 'region']

raw_data = pd.get_dummies(raw_data, columns=qualitative_columns)
raw_data = raw_data.drop(columns=['smoker_no'])

In [11]:
data = raw_data

In [12]:
data.head()

,age,bmi,children,charges,smoker_yes,sex_female,sex_male,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,1,0,0,0,0,1
1,18,33.770,1,1725.55230,0,0,1,0,0,1,0
2,28,33.000,3,4449.46200,0,0,1,0,0,1,0
3,33,22.705,0,21984.47061,0,0,1,0,1,0,0
4,32,28.880,0,3866.85520,0,0,1,0,1,0,0


In [13]:
data['charges'].hist(grid=True, bins=20)

In [14]:
# regression

In [15]:
# 1 - searching for best regressor

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

In [17]:
parameters = {
    'splitter':['best', 'random'], 
    'min_samples_split':[2, 3, 4], 
    'max_features':['auto', 'sqrt', 'log2', None]
}

dt_model = DecisionTreeRegressor()

gscv = GridSearchCV(dt_model, parameters, cv=5, iid=False)
gscv.fit(data.drop(columns=['charges']), data['charges'])

print('best features:', gscv.best_params_)
print('best score:', gscv.best_score_)

best features: {'max_features': 'auto', 'min_samples_split': 4, 'splitter': 'random'}
best score: 0.7567504119930041


In [18]:
# 2 - estimating best regressors accuracy

In [19]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [20]:
train, test = train_test_split(data, test_size=0.3)

best_model = DecisionTreeRegressor(splitter='random', min_samples_split=4, max_features='auto')
best_model.fit(train.drop(columns=['charges']), train['charges'])

y_pred = best_model.predict(test.drop(columns=['charges']))

In [21]:
print('r-squared:', best_model.score(test.drop(columns=['charges']), test['charges']))
print('mse:', mean_squared_error(test['charges'], y_pred))

r-squared: 0.7340161520628574
mse: 40135682.34129627


In [ ]:
# model coefficients - importance

In [47]:
cols = list(data.columns)
cols.remove('charges')

tmp = pd.DataFrame(data=best_model.feature_importances_, columns=cols)
tmp.head()

ValueError: Shape of passed values is (10, 1), indices imply (10, 10)

In [22]:
data.head()

,age,bmi,children,charges,smoker_yes,sex_female,sex_male,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,1,0,0,0,0,1
1,18,33.770,1,1725.55230,0,0,1,0,0,1,0
2,28,33.000,3,4449.46200,0,0,1,0,0,1,0
3,33,22.705,0,21984.47061,0,0,1,0,1,0,0
4,32,28.880,0,3866.85520,0,0,1,0,1,0,0


In [31]:
print('Estimated cost of insurance:', round(best_model.predict(np.array(data_array).reshape(1, -1))[0], 2), '$')

Estimated cost of insurance: 34471.72 $
